In [422]:
import numpy as np
import pandas as pd

In [423]:
movie_data = pd.read_csv('/content/movies.csv')
rating_data = pd.read_csv('/content/ratings.csv')

In [424]:
movie_data.head(6)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller


In [425]:
movie_data['year'] = movie_data.title.str.extract('(\(\d\d\d\d\))',expand=False)
movie_data['year']

0        (1995)
1        (1995)
2        (1995)
3        (1995)
4        (1995)
          ...  
62418    (2018)
62419    (2001)
62420    (2018)
62421    (2001)
62422    (1962)
Name: year, Length: 62423, dtype: object

In [426]:
movie_data['year'] = movie_data.year.str.extract('(\d\d\d\d)',expand=False)
movie_data['year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
62418    2018
62419    2001
62420    2018
62421    2001
62422    1962
Name: year, Length: 62423, dtype: object

In [427]:
movie_data['title'] = movie_data.title.str.replace('(\(\d\d\d\d\))', '')
movie_data['title']
movie_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [428]:
movie_data['title'] = movie_data['title'].apply(lambda x: x.strip())

In [429]:
movie_data = movie_data.drop('genres', 1)
movie_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [430]:
rating_data.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [431]:
ratings_df = rating_data.drop('timestamp', 1)
ratings_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [432]:
user_input_data = [
            {'title':'Toy Story', 'rating':4},
            {'title':'Waiting to Exhale', 'rating':2.5},
            {'title':'Jumanji', 'rating':3.5},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':2},
         ] 
input_movies = pd.DataFrame(user_input_data)
input_movies

,title,rating
0,Toy Story,4.0
1,Waiting to Exhale,2.5
2,Jumanji,3.5
3,Pulp Fiction,5.0
4,Akira,2.0


In [433]:
inputId = movie_data[movie_data['title'].isin(input_movies['title'].tolist())]
inputMovies = pd.merge(inputId, input_movies)
inputMovies = inputMovies.drop('year', 1)
inputMovies

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,title,rating
0,1,Toy Story,4.0
1,2,Jumanji,3.5
2,4,Waiting to Exhale,2.5
3,296,Pulp Fiction,5.0
4,1274,Akira,2.0
5,164600,Akira,2.0


In [434]:
user_set = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
user_set.head()

,userId,movieId,rating
0,1,296,5.0
70,2,1,3.5
254,3,1,4.0
264,3,296,5.0
299,3,1274,4.5


In [435]:
user_group = user_set.groupby(['userId'])

In [436]:
user_group.get_group(120)

,userId,movieId,rating
15053,120,1,5.0
15082,120,296,5.0


In [437]:
user_group = sorted(user_group,  key=lambda x: len(x[1]), reverse=True)

In [438]:
user_group[0]

(1977,         userId  movieId  rating
 283097    1977        1     4.5
 283098    1977        2     4.0
 283100    1977        4     1.0
 283160    1977      296     4.5
 283408    1977     1274     2.5)

In [439]:
user_group[0][0]

1977

In [440]:
user_group[0][1]

,userId,movieId,rating
283097,1977,1,4.5
283098,1977,2,4.0
283100,1977,4,1.0
283160,1977,296,4.5
283408,1977,1274,2.5


In [441]:
user_group = user_group[0:100]

In [442]:
pearsonCorrelationDict = {}
for name, group in user_group:
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    nRatings = len(group)
    temp_data = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    temp_rlist = temp_data['rating'].tolist()
    temp_glist = group['rating'].tolist()
    Sxx = sum([i**2 for i in temp_rlist]) - pow(sum(temp_rlist),2)/float(nRatings)
    Syy = sum([i**2 for i in temp_glist]) - pow(sum(temp_glist),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(temp_rlist, temp_glist)) - sum(temp_rlist)*sum(temp_glist)/float(nRatings)
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/np.sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [443]:
pearsonCorrelationDict.items()

dict_items([(1977, 0.81035088270649), (6747, 0.5984873720496731), (8619, 0.49114974312875137), (8735, 0.6494998689336409), (9022, 0.6007514091861108), (10493, 0.5789473684210519), (11952, 0.9136323139725234), (15121, 0.2663354049093592), (15938, 0.4959348919001857), (16222, 0.9762210399274297), (18306, 0.727157461958315), (22384, 0.8673649643743675), (23128, 0.34341479757995474), (28914, 0.6264302520481555), (29344, -0.9551338658818386), (30643, 0.863294071085508), (31207, 0.6061426456557821), (31567, -0.42749098003548075), (32290, 0.42755550210664106), (33047, 0.24856567949974787), (35368, 0.30588764516074796), (35821, 0.6061426456557814), (35963, 0.3980489699222203), (40216, 0.2221308291596596), (40333, 0.8411910241920598), (42029, 0.1962278122357475), (42718, 0.8816620300447731), (46053, 0.13394992994420493), (47370, 0.7404360971988653), (47628, -0.49793173880750663), (48845, 0.7870533935989636), (49403, 0.45166178498192183), (49629, 0.5429864712791679), (51678, 0.7404360971988653),

In [444]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.head()

,0
1977,0.810351
6747,0.598487
8619,0.491150
8735,0.649500
9022,0.600751


In [445]:
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.810351,1977
1,0.598487,6747
2,0.491150,8619
3,0.649500,8735
4,0.600751,9022


In [446]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head(20)

,similarityIndex,userId
94,0.984495,3394
68,0.980581,1570
9,0.976221,16222
41,0.942421,60305
56,0.919866,836
6,0.913632,11952
62,0.907393,1140
26,0.881662,42718
67,0.870388,1436
45,0.870388,171


In [447]:
topUsersRating = topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.984495,3394,1,4.0
1,0.984495,3394,2,3.5
2,0.984495,3394,3,1.0
3,0.984495,3394,4,3.0
4,0.984495,3394,6,3.0


In [448]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.984495,3394,1,4.0,3.937981
1,0.984495,3394,2,3.5,3.445733
2,0.984495,3394,3,1.0,0.984495
3,0.984495,3394,4,3.0,2.953486
4,0.984495,3394,6,3.0,2.953486


In [449]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,34.788227,139.392642
2,34.364568,105.558881
3,22.222504,60.031458
4,23.973035,50.879229
5,17.154378,40.390616


In [450]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,4.006891,1
2,3.071736,2
3,2.701381,3
4,2.122352,4
5,2.354537,5


In [451]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
134849,5.0,134849
4495,5.0,4495
167762,5.0,167762
171271,5.0,171271
3096,5.0,3096


In [452]:
movie_data.loc[movie_data['movieId'].isin(recommendation_df.head(20)['movieId'].tolist())]

,movieId,title,year
2821,2913,The Mating Habits of the Earthbound Human,1999
2976,3069,Effect of Gamma Rays on Man-in-the-Moon Marigo...,1972
3003,3096,My Man Godfrey,1957
4390,4495,Crossing Delancey,1988
8638,26184,"Diamond Arm, The (Brilliantovaya ruka)",1968
29848,134849,Duck Amuck,1953
42571,164147,Behind the Planet of the Apes,1998
42748,164540,Amanda Knox,2016
43193,165483,Joe Rogan: Triggered,2016
44219,167762,Batman Beyond Darwyn Cooke's Batman 75th Anniv...,2014


In [453]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
rmse_val = rmse(np.array(topUsersRating['weightedRating']), np.array(topUsersRating['rating']))
print("RMSE Value is: " + str(rmse_val))

RMSE Value is: 1.1546368996722423
